### Anonymisation



In [ ]:
# Add to system path so we can find all the packages
import sys
sys.path
sys.path.append('C:\\Program Files\\Python37\\Lib\\site-packages')
sys.path.append('C:\Program Files\Python37')

# Load user-written functions

get_ipython().run_line_magic('load_ext', 'autoreload')
get_ipython().run_line_magic('autoreload', '2')

#import text_functions

In [ ]:
import os
os.chdir("C:\\Users\\[username]\\Downloads\\Updated Text Data") # insert [username]

### Identify personal information to delete

In [ ]:
## Identify names of parents and carers for anonymisation
# Run once and then just import the pickled data as it takes a while
assert 1==2

import numpy as np
import re
import spacy
from spacy.lang.en import English
import en_core_web_sm
nlp = en_core_web_sm.load()

parent_carers_names = pd.read_excel('My Documents07754.xlsx')
parent_carers_names = parent_carers_names[['PSID','ShortAnswer']]

# ShortAnswer has the possiblity of including professionals but they are all missing
#parents_carers = parent_carers_names.loc[parent_carers_names['SecondaryQuestion']=="Name of parent(s)/carer(s)",'ShortAnswer']
parent_carers_names['ShortAnswer'] = [name.strip() if not type(name)==float else np.nan for name in parent_carers_names['ShortAnswer']]

generic_words = (['Not applicable', 'Mother', 'Father', 'Both parents', 'and', 
                  'Grandmother', 'Grandfather', 'Aunt', 'Uncle', 'Stepfather', 'Stepmother',
                    'Ms', 'Mr', 'Mrs', '&', ''])

for word in generic_words:
    parent_carers_names['ShortAnswer'] = [re.sub(word, '', name, flags=re.IGNORECASE) if not type(name)==float else np.nan for name in parent_carers_names['ShortAnswer']]
    parent_carers_names['ShortAnswer'] = [re.sub(r"[^\w\s]", "", name) if not type(name)==float else np.nan for name in parent_carers_names['ShortAnswer']]

# Some involve full sentences including information about the parents - just want their names
# Identify Proper nouns using Spacy
parent_carers_names['Parent / Carer'] = [[] for i in range(parent_carers_names.shape[0])]
for idx in parent_carers_names['ShortAnswer'].index:
    string = parent_carers_names['ShortAnswer'][idx]
    if string == string:
        doc = nlp(string)
        prop_noun = list(set([token.text for token in doc if token.pos_ == "PROPN"]))
        #print(string)
        #print(prop_noun)
        parent_carers_names.at[idx, 'Parent / Carer'] = prop_noun

# One row by PSID with all parents, carers names
parent_carers_names[['PSID','Parent / Carer']] = parent_carers_names[['PSID','Parent / Carer']].groupby('PSID', as_index = False).agg(sum) 
parent_carers_names['Parent / Carer name'] = [list(set(p_c)) if type(p_c) != float else np.nan for p_c in parent_carers_names['Parent / Carer']]

# Save names separately to use in anonymisation
import pickle
with open("Other\\Parent Carer Names.pkl", "wb") as handle:
    pickle.dump(parent_carers_names[['PSID','Parent / Carer name']], handle, protocol = pickle.HIGHEST_PROTOCOL)


In [ ]:
import pandas as pd
import string

# Import first and last names of CYP
first_names = pd.read_csv("Other\\First.csv", encoding = 'latin')
last_names = pd.read_csv("Other\\Last.csv", encoding = 'latin')

# Combine to form a list of unique first and last names
first_names_list = [str(name).split(' ') for name in list(first_names['ClientName.1'])]
first_names_list = [name for sublist in first_names_list for name in sublist]
first_names_list = [name for name in first_names_list if name not in string.punctuation]
first_names_list = [name for name in first_names_list if not isinstance(name, int)]
first_names_list  = [name.capitalize() for name in first_names_list if type(name)!= float]
first_names_list = list(set(first_names_list))
print(len(first_names_list))
last_names_list = [str(name).split(' ') for name in list(last_names['ClientName.2'])]
last_names_list = [name for sublist in last_names_list for name in sublist]
last_names_list = [name for name in last_names_list if name not in string.punctuation]
last_names_list = [name for name in last_names_list if not isinstance(name, int)]
last_names_list  = [name.capitalize() for name in last_names_list if type(name)!= float]
last_names_list = list(set(last_names_list))
print(len(last_names_list))
all_names = list(set(first_names_list).union(set(last_names_list)))
all_names = [name for name in all_names if '(' not in name] # As otherwise loop bugs
all_names = [name for name in all_names if ')' not in name] 
print(len(all_names))

In [ ]:
# Bring in list of baby names so that we can get rid of other names 
# List of baby names (1996-2017, England and Wales) from ONS - most comprehensive list available
import pandas as pd
baby_names_boys = pd.read_excel("Other\\adhocallbabynames1996to2017.xls", sheet_name = "Boys", skiprows = 5)
baby_names_girls = pd.read_excel("Other\\adhocallbabynames1996to2017.xls", sheet_name = "Girls", skiprows = 5)
list_baby_names = list(set(list(baby_names_girls["Name"]) + list(baby_names_boys["Name"])))
# Strip, make sentence case
list_baby_names = [name.strip() for name in list_baby_names if type(name)!= float]
list_baby_names = [name.capitalize() for name in list_baby_names if type(name)!= float]
list_baby_names = [name for name in list_baby_names if len(name) >=2 if type(name)!= float]
print(len(list_baby_names))
# Delete those in the dataset already
list_baby_names = list(set(list_baby_names).difference(set(all_names)))
print(len(list_baby_names))

In [ ]:
# Use team names to find local places
import numpy as np
import pandas as pd
import re
team_name = pd.read_csv("Other\\Team Names.csv")
place_names = [name for name in list(team_name['Team Name']) if 'SU' in name]
place_names = [re.sub(r'SU[0-9]{2} ', '', name) for name in place_names]
place_names.extend(["xxx"]) # Obscured for anonymity of the LA

# Using place information from schools spreadsheet
schools = pd.read_csv("Other\\2017-2018_860_spine.csv")
place_names.extend(list(schools['LOCALITY']))
place_names.extend(list(schools['TOWN']))
place_names = list(set(place_names))
place_names = [name for name in place_names if name is not np.nan]

In [ ]:
import numpy as np
import pandas as pd
import re

# Add school names
schools = pd.read_csv("Other\\2017-2018_860_spine.csv")
school_names = list(set(list(schools['SCHNAME']) + list(schools['OTHERSCHNAME'])))
school_names = [name for name in school_names if name is not np.nan]

school_pattern = re.compile(r""" Primary School| Primary Academy| First School| Academy|
                            Infant School Academy| School| Junior School| High School and Sixth Form
                            | High School| Middle School| College| Infants' School| Infant Academy|
                             Infant School|  Nursery| Sport and Community| Community Infants and Nursery
                             | Catholic School| Training School| Community Special| Controlled School
                             | Catholic Voluntary School| Sixth Form| Primary and Nursery| Technology School| Community School| Secondary School| Preparatory School|
                              Community Infants and Nursery| Infant & Nursery|  Infant  Nursery| Independent School|
                               Sports| Grammar""")

school_names = [re.sub(school_pattern, "", name) for name in school_names]
school_names = [re.sub("\([A-Z]+\)", "", name) for name in school_names] # Getting rid of "(A)" or "(C)"
school_names_no_punc = [re.sub(r"[^\w\s]", "", name) for name in school_names] # Creating another list with no punctuation
school_names.extend(school_names_no_punc) 
school_names = [name.strip() for name in school_names]
school_names = list(set(school_names))
print(len(school_names))

In [ ]:
# Social workers may write school names in different ways
# Create variations for C of E schools
school_names_cofe = [name for name in school_names if ('CE' in name) or ('CofE' in name) or ('Church of England' in name)]
school_names_cofe_all_variations = []
for school in school_names_cofe:
    variation_list = ["CE", "CofE", "Church of England"]
    for variation in variation_list:
        if variation in school:
            variation_list.remove(variation)
            school_names_cofe_all_variations.append(school)
            for var in variation_list:
                sch = re.sub(variation, var, school) 
                school_names_cofe_all_variations.append(sch)

school_names.extend(school_names_cofe_all_variations)
school_names_st = [name for name in school_names if ('St ' in name) or ('ST ' in name) or ('St. ' in name) or ('ST. ' in name) or ('Saint ' in name) or ('SAINT ' in name)]

# Create variations for schools with St in name
school_names_st_all_variations = []
for school in school_names_st:
    variation_list = ['St ', 'ST ', 'St. ', 'ST. ', 'Saint ', 'SAINT ']
    for variation in variation_list:
        if variation in school:
            variation_list.remove(variation)
            school_names_st_all_variations.append(school)
            for var in variation_list:
                sch = re.sub(variation, var, school) 
                school_names_st_all_variations.append(sch)
                
school_names.extend(school_names_st_all_variations)
school_names = list(set(school_names))
print(len(school_names))

### Replace known personal information with placeholders

In [ ]:
import os
import pickle

os.chdir("C:\\Users\\[username]\\Downloads\\Updated Text Data")  # insert [username]
filename = open("Created\\rq2data_text_for_anonymisation.pkl", "rb")
text = pickle.load(filename)

text_cols = (['Child Social Work Assessment for Review Child Protection Conference_text',
       'Child Social Work Assessment to Initial Child Protection Conference_text',
       'Child Social Work Assessment_text', 'Contact and Referral Form_text',
       'Child Social Work Assessment for Review Child Protection Conference_text_prev',
       'Child Social Work Assessment to Initial Child Protection Conference_text_prev',
       'Child Social Work Assessment_text_prev',
       'Contact and Referral Form_text_prev'])

# Drop nas
print(text.shape)
print(text.isna().sum())
text.dropna(subset = text_cols, how = 'all', inplace = True) # if all text is blank
text[text_cols] = text[text_cols].fillna('')
print(text.shape)
print(text.isna().sum())

In [ ]:
import os
import pickle
import pandas as pd

# Parent / carer names
os.chdir("C:\\Users\\[username]\\Downloads\\Updated Text Data")  # insert [username]
filename = open("Other\\Parent Carer Names.pkl", "rb")
parent_carer_names_df = pickle.load(filename)

# Merge into text data
print(text.shape)
text = text.merge(parent_carer_names_df, how = 'left', on = 'PSID')
print(text.shape)
print(text['Parent / Carer name'].isna().sum())

In [ ]:
# Replace available names with placeholders
import pickle
import numpy as np
import re

## Record how many replacements to keep track of progress
# _replaced = total number replaced, _n = number replaced in that document
# documents_changed_ = total number of documents changed
names_replaced, places_replaced, schools_replaced, parents_carers_names_replaced = 0, 0, 0, 0
documents_changed_names, documents_changed_places, documents_changed_schools, documents_changed_p_c_names = 0, 0, 0, 0

save_every = 100

text_cols = (['Child Social Work Assessment_text']) # 14/01 morning

for text_col in text_cols:
    print("Column: ", text_col)
    already_done_list = []
    for idx in text.index[4300:]:        
        print("Idx: ", idx)
        string = text.loc[idx, text_col]
        #print(string)
        if (string.strip() != '') and (idx not in already_done_list):
            # Dealing with formatting issues
            # Strip space characters
            string = re.sub(r'\W+', ' ', str(string))
            schools_n = 0
            for school in school_names:
                stringn = re.subn(r"\b{}\b".format(school), "School_Name", string, flags = re.IGNORECASE)
                string = stringn[0]
                schools_n += stringn[1]  
            names_n = 0
            for name in all_names:
                stringn = re.subn(r"\b{}\b".format(name), "Name", string, flags = re.IGNORECASE)
                string = stringn[0]
                names_n +=stringn[1]
            places_n = 0
            for place in place_names:
                stringn = re.subn(r"\b{}\b".format(place), "Location", string, flags = re.IGNORECASE)
                string = stringn[0]
                places_n += stringn[1]              
            parents_carers_names = text.loc[idx, "Parent / Carer name"]
            #print("Parents / carers: ", parents_carers_names)
            parents_carers_names_n = 0
            if type(parents_carers_names) != float: 
                for name in parents_carers_names:
                    stringn = re.subn(r"\b{}\b".format(name), "Parent_Carer_Name", string, flags = re.IGNORECASE)
                    string = stringn[0]
                    parents_carers_names_n += stringn[1]                      
            repeated_text_indices = list(np.where(text[text_col] ==  text.loc[idx,text_col])[0])
            print(repeated_text_indices)
            print("Names n: ", names_n)
            print("Places n: ", places_n)
            print("Schools n: ", schools_n)
            print("Parent carers n: ", parents_carers_names_n)
            if repeated_text_indices:
                print(repeated_text_indices)
                text.at[repeated_text_indices,text_col] = string
                already_done_list.extend(repeated_text_indices)
                names_replaced += (len(repeated_text_indices) * names_n)
                places_replaced += (len(repeated_text_indices) * places_n)
                schools_replaced += (len(repeated_text_indices) * schools_n)
                parents_carers_names_replaced += (len(repeated_text_indices) * parents_carers_names_n)
                if names_n != 0:
                    documents_changed_names += len(repeated_text_indices)
                    print("Number of documents where names were changed: ", documents_changed_names)
                if places_n != 0:
                    documents_changed_places += len(repeated_text_indices)
                    print("Number of documents where places were changed: ", documents_changed_places)
                if schools_n != 0:
                    documents_changed_schools += len(repeated_text_indices)
                    print("Number of documents where school names were changed: ", documents_changed_schools)
                if parents_carers_names_n != 0:
                    documents_changed_p_c_names += len(repeated_text_indices)
                    print("Number of documents where parents' / carers' names were changed: ", documents_changed_p_c_names)
            else:
                text.at[idx,text_col] = string
                names_replaced += names_n
                places_replaced += places_n
                schools_replaced += schools_n
                parents_carers_names_replaced += parents_carers_names_n
                if names_n != 0:
                    documents_changed_names += 1
                    print("Number of documents where names were changed: ", documents_changed_names)
                if places_n != 0:
                    documents_changed_places += 1
                    print("Number of documents where places were changed: ", documents_changed_places)
                if schools_n != 0:
                    documents_changed_schools += 1
                    print("Number of documents where school names were changed: ", documents_changed_schools)
                if parents_carers_names_n != 0:
                    documents_changed_p_c_names += 1
                    print("Number of documents where parents' / carers' names were changed: ", documents_changed_p_c_names)
        if (idx % save_every == 0) & (idx != 0):
            with open("Created/Anonymisation/text_rq2_names_placeholders_{}_{}_{}.pkl".format(text_col, idx - save_every, idx - 1), "wb") as handle:
                pickle.dump(text.loc[idx-save_every: idx - 1, text_col], handle, protocol = pickle.HIGHEST_PROTOCOL)   
        elif (idx == text.shape[0] - 1):
            with open("Created/Anonymisation/text_rq2_names_placeholders_{}_{}_{}.pkl".format(text_col, idx - idx % save_every - 1, idx), "wb") as handle:
                pickle.dump(text.loc[idx - idx % save_every: idx, text_col], handle, protocol = pickle.HIGHEST_PROTOCOL)   
                 

In [ ]:
# Knit together files
import glob
import os
import pickle
import pandas as pd
import re

os.chdir("C:\\Users\\[username]\\Downloads\\Updated Text Data")  # insert [username]
filename_list = [file for file in glob.glob("Created\\Anonymisation\\text_rq2_names_placeholders_*.pkl")]

# Call in all files in the anonymisation folder
file_dict = {}
for file in filename_list:
    filename = open(file, "rb")
    f = pickle.load(filename)
    file_n = re.sub('.pkl', '', file)
    file_n = re.sub("Created\\\\Anonymisation\\\\text_rq2_names_placeholders_", '', file_n)
    print(file_n)
    file_dict[file_n] = f


text_cols = (['Child Social Work Assessment for Review Child Protection Conference_text',
       'Child Social Work Assessment to Initial Child Protection Conference_text',
       'Contact and Referral Form_text',
       'Child Social Work Assessment for Review Child Protection Conference_text_prev',
       'Child Social Work Assessment to Initial Child Protection Conference_text_prev',
        'Child Social Work Assessment_text', 'Child Social Work Assessment_text_prev',
       'Contact and Referral Form_text_prev']) 

col_wo_identifier_dict = {}
for text_col in text_cols:
    text_col_df = [df for df in file_dict.keys() if text_col in df]
    if 'prev' in text_col:
        text_col_df = [df for df in text_col_df if 'prev' in df]
    else:
        text_col_df = [df for df in text_col_df if 'prev' not in df]
    print(text_col_df)
    text_col_df_values = [v for k, v in file_dict.items() if k in text_col_df]
    print(len(text_col_df_values))
    col_wo_identifier_dict[text_col] = pd.concat(text_col_df_values, axis = 0)
    col_wo_identifier_dict[text_col].sort_index(inplace = True)
    

In [ ]:
### Correct misaligned indices

# 'Child Social Work Assessment for Review Child Protection Conference_text' - fine
# Create new columns with same name but suffix 2 so they don't drop out below
col_wo_identifier_dict['Child Social Work Assessment for Review Child Protection Conference_text_2'] = col_wo_identifier_dict['Child Social Work Assessment for Review Child Protection Conference_text'] 
col_wo_identifier_dict['Child Social Work Assessment_text_2'] = col_wo_identifier_dict['Child Social Work Assessment_text'] 

col_wo_identifier_dict['Child Social Work Assessment to Initial Child Protection Conference_text_2'] = pd.Series(index = col_wo_identifier_dict['Child Social Work Assessment to Initial Child Protection Conference_text'].index)
col_wo_identifier_dict['Child Social Work Assessment to Initial Child Protection Conference_text_2'].loc[10045:] = col_wo_identifier_dict['Child Social Work Assessment to Initial Child Protection Conference_text'].loc[10045:]
col_wo_identifier_dict['Child Social Work Assessment to Initial Child Protection Conference_text_2'] = col_wo_identifier_dict['Child Social Work Assessment to Initial Child Protection Conference_text'].shift(1)
col_wo_identifier_dict['Child Social Work Assessment to Initial Child Protection Conference_text_2'].loc[:10044] = col_wo_identifier_dict['Child Social Work Assessment to Initial Child Protection Conference_text'].loc[:10044]
# col_wo_identifier_dict['Child Social Work Assessment to Initial Child Protection Conference_text_2'].loc[9653:10046,]

# 10038 now missing instead of text but ok for now
col_wo_identifier_dict['Contact and Referral Form_text_2'] = pd.Series(index = col_wo_identifier_dict['Contact and Referral Form_text'].index)
col_wo_identifier_dict['Contact and Referral Form_text_2'].loc[10038:] = col_wo_identifier_dict['Contact and Referral Form_text'].loc[10038:]
col_wo_identifier_dict['Contact and Referral Form_text_2'] = col_wo_identifier_dict['Contact and Referral Form_text_2'].shift(1)
col_wo_identifier_dict['Contact and Referral Form_text_2'].loc[:10036] = col_wo_identifier_dict['Contact and Referral Form_text'].loc[:10036]

#col_wo_identifier_dict['Contact and Referral Form_text_2'].loc[10035:10045,]

# 10038 now missing instead of text but ok for now
col_wo_identifier_dict['Child Social Work Assessment for Review Child Protection Conference_text_prev_2'] = pd.Series(index = col_wo_identifier_dict['Child Social Work Assessment for Review Child Protection Conference_text_prev'].index)
col_wo_identifier_dict['Child Social Work Assessment for Review Child Protection Conference_text_prev_2'].loc[10038:] = col_wo_identifier_dict['Child Social Work Assessment for Review Child Protection Conference_text_prev'].loc[10038:]
col_wo_identifier_dict['Child Social Work Assessment for Review Child Protection Conference_text_prev_2'] = col_wo_identifier_dict['Child Social Work Assessment for Review Child Protection Conference_text_prev_2'].shift(1)
col_wo_identifier_dict['Child Social Work Assessment for Review Child Protection Conference_text_prev_2'].loc[:10036] = col_wo_identifier_dict['Child Social Work Assessment for Review Child Protection Conference_text_prev'].loc[:10036]

#col_wo_identifier_dict['Child Social Work Assessment for Review Child Protection Conference_text_prev_2'].loc[10035:10045,]
#col_wo_identifier_dict['Child Social Work Assessment for Review Child Protection Conference_text_prev_2'].tail()

col_wo_identifier_dict['Child Social Work Assessment to Initial Child Protection Conference_text_prev_2'] = pd.Series(index = col_wo_identifier_dict['Child Social Work Assessment to Initial Child Protection Conference_text_prev'].index)
col_wo_identifier_dict['Child Social Work Assessment to Initial Child Protection Conference_text_prev_2'].loc[10038:] = col_wo_identifier_dict['Child Social Work Assessment to Initial Child Protection Conference_text_prev'].loc[10038:]
col_wo_identifier_dict['Child Social Work Assessment to Initial Child Protection Conference_text_prev_2'] = col_wo_identifier_dict['Child Social Work Assessment to Initial Child Protection Conference_text_prev_2'].shift(1)
col_wo_identifier_dict['Child Social Work Assessment to Initial Child Protection Conference_text_prev_2'].loc[:10036] = col_wo_identifier_dict['Child Social Work Assessment to Initial Child Protection Conference_text_prev'].loc[:10036]

#col_wo_identifier_dict['Child Social Work Assessment to Initial Child Protection Conference_text_prev_2'].loc[10035:10045,]
#col_wo_identifier_dict['Child Social Work Assessment to Initial Child Protection Conference_text_prev_2'].tail()

# 10038 now missing instead of text but ok for now; 11457 is missing
col_wo_identifier_dict['Child Social Work Assessment_text_prev_2'] = pd.Series(index = col_wo_identifier_dict['Child Social Work Assessment_text_prev'].index)
col_wo_identifier_dict['Child Social Work Assessment_text_prev_2'].loc[10038:] = col_wo_identifier_dict['Child Social Work Assessment_text_prev'].loc[10038:]
col_wo_identifier_dict['Child Social Work Assessment_text_prev_2'] = col_wo_identifier_dict['Child Social Work Assessment_text_prev_2'].shift(1)
col_wo_identifier_dict['Child Social Work Assessment_text_prev_2'].loc[:10036] = col_wo_identifier_dict['Child Social Work Assessment_text_prev'].loc[:10036]

#col_wo_identifier_dict['Child Social Work Assessment_text_prev_2'].loc[10035:10045,]
#col_wo_identifier_dict['Child Social Work Assessment_text_prev_2'].tail()

# 10038 now missing instead of text but ok for now; 11457 is missing
col_wo_identifier_dict['Contact and Referral Form_text_prev_2'] = pd.Series(index = col_wo_identifier_dict['Contact and Referral Form_text_prev'].index)
col_wo_identifier_dict['Contact and Referral Form_text_prev_2'].loc[10038:] = col_wo_identifier_dict['Contact and Referral Form_text_prev'].loc[10038:]
col_wo_identifier_dict['Contact and Referral Form_text_prev_2'] = col_wo_identifier_dict['Contact and Referral Form_text_prev_2'].shift(1)
col_wo_identifier_dict['Contact and Referral Form_text_prev_2'].loc[:10036] = col_wo_identifier_dict['Contact and Referral Form_text_prev'].loc[:10036]

#col_wo_identifier_dict['Contact and Referral Form_text_prev_2'].loc[10035:10045,]
#col_wo_identifier_dict['Contact and Referral Form_text_prev_2'].tail()

In [ ]:
from functools import reduce

# Deduplicate by index (in case any rows are repeated in different files)
for col, df in col_wo_identifier_dict.items():
    print(col)
    print(df.shape)
    col_wo_identifier_dict[col] = df.loc[~df.index.duplicated(keep = 'first')]
    print(col_wo_identifier_dict[col].shape)
    col_wo_identifier_dict[col].rename(col, inplace = True)
    
# Merge together on index
df_wo_identifier_together = reduce(lambda left, right: pd.merge(left,right,left_index = True, right_index = True, how = 'outer'), col_wo_identifier_dict.values())
df_wo_identifier_together = df_wo_identifier_together.loc[~df_wo_identifier_together.index.duplicated(keep = 'first')]
print(sorted(df_wo_identifier_together.columns))

In [ ]:
# Rename adjusted columns to have the same column names as expected
print(df_wo_identifier_together.columns)
df_wo_identifier_together.drop(columns = text_cols, inplace = True)
print(df_wo_identifier_together.columns)

original_rename_dict = dict(zip(sorted(df_wo_identifier_together.columns), sorted(text_cols)))
print(original_rename_dict)

df_wo_identifier_together.rename(columns = original_rename_dict, inplace = True)
print(df_wo_identifier_together.columns)

In [ ]:
# Bring in the original text dataset for anonymisation so that we can merge in the IDs
# Reset index so the text lines up df_wo_identifier_together
# The problem was a combination of the indices starting at 7221, 7222, 0, 1, 2 etc and missing data

import os
import pickle

os.chdir("C:\\Users\\[username]\\Downloads\\Updated Text Data")  # insert [username]
filename = open("Created\\rq2data_text_for_anonymisation.pkl", "rb")
text = pickle.load(filename)
print(text.index)
text.reset_index(drop = True, inplace = True) # to line up with df_wo_identifier_together
print(text.index)

# Merge back in IDs
print(text.shape)
print(text.columns)
text = pd.merge(text[['PSID', 'ReferralDatetime']], df_wo_identifier_together, left_index = True, right_index = True, how = 'left')
print(text.shape)
print(text.columns)

# Replace missing with empty string

print(text.isna().sum()) 
text[text_cols] = text[text_cols].fillna('')
print(text.isna().sum())


In [ ]:
# Save knitted data all together 
with open("Created/text_rq2_names_placeholders.pkl", "wb") as handle:
    pickle.dump(df_wo_identifier_together, handle, protocol = pickle.HIGHEST_PROTOCOL) # TO CORRECT: should be text

In [ ]:
#!pip install spacy
#!python -m spacy download  en_core_web_sm

In [ ]:
# Getting rid of other names by fuzzy matching on baby name list
import difflib
import numpy as np
import pickle
import re
import spacy
from spacy.lang.en import English
import en_core_web_sm
nlp = en_core_web_sm.load()

filename = open("Created/text_rq2_names_placeholders.pkl", "rb")
text = pickle.load(filename)
print(text.isna().sum())
text.fillna('', inplace = True)
print(text.isna().sum())

print(text.shape)
print(text.index)
print(text.columns)

not_names = (['Name'])
save_every = 100

text_cols = (['Child Social Work Assessment_text'])
    

# _replaced = total number replaced, _n = number replaced in that document
# documents_changed_ = total number of documents changed
other_names_replaced = 0
documents_changed_other_names = 0

nlp = en_core_web_sm.load()
text2 = pd.DataFrame(None, index = text.index, columns = text_cols, dtype = 'object')
fuzzy_dict = {}
for text_col in text_cols:
    print("Column: ", text_col)
    already_done_list = []
    for idx in text.index[400:]:
        print("Idx: ", idx)
        string = text.loc[idx, text_col]
        string = re.sub('\(|\)|\*', '', string)
        #print("String: ", string[0:20])
        if (string.strip() != '') and (idx not in already_done_list):
            doc = nlp(string)
            prop_noun = list(set([token.text for token in doc if token.pos_ == "PROPN"]))
            print("Prop_noun: ", prop_noun)
            other_names_n = 0
            for p in prop_noun:
                if p not in not_names:
                    fuzzies_matched = fuzzy_dict.get(p, None)
                    if fuzzies_matched is None:
                        fuzzy_names = difflib.get_close_matches(p.capitalize(), list_baby_names, cutoff=0.9)
                        print(p, fuzzy_names)
                        fuzzy_dict[p] = fuzzies_matched
                    if fuzzies_matched != []:
                        stringn = re.subn(r"\b{}\b".format(p), 'Other_Name', string, flags = re.IGNORECASE)
                        string = stringn[0]
                        other_names_n +=stringn[1]
            repeated_text_indices = list(np.where(text[text_col] == text.loc[idx, text_col])[0])
            print(repeated_text_indices)
            if repeated_text_indices:
                print(repeated_text_indices)
                text2.loc[repeated_text_indices, text_col] = string
                already_done_list.extend(repeated_text_indices) 
                other_names_replaced += (len(repeated_text_indices) * other_names_n)
                if other_names_n != 0:
                    documents_changed_other_names += len(repeated_text_indices)
                    print("Number of documents where other names were changed: ", documents_changed_other_names)
            else:
                text2.at[idx, text_col] = string
                other_names_replaced += other_names_n
                if other_names_n != 0:
                    documents_changed_other_names += 1
                    print("Number of documents where other names were changed: ", documents_changed_other_names)
        if (idx % save_every == 0) & (idx != 0):
            with open("Created/Anonymisation/text_rq2_names_matched_{}_{}_{}.pkl".format(text_col, idx - save_every, idx - 1), "wb") as handle:
                pickle.dump(text2.loc[idx-save_every: idx - 1, text_col], handle, protocol = pickle.HIGHEST_PROTOCOL)   
        elif (idx == text.shape[0] - 1):
            with open("Created/Anonymisation/text_rq2_names_matched_{}_{}_{}.pkl".format(text_col, idx - idx % save_every - 1, idx), "wb") as handle:
                pickle.dump(text2.loc[idx - idx % save_every: idx, text_col], handle, protocol = pickle.HIGHEST_PROTOCOL)          

with open("Created/text_rq2_names_matched.pkl", "wb") as handle:
    pickle.dump(text2, handle, protocol = pickle.HIGHEST_PROTOCOL) 

In [ ]:
# Knit together files
import glob
import os
import pickle
import pandas as pd
import re

os.chdir("C:\\Users\\[username]\\Downloads\\Updated Text Data")  # insert [username]
filename_list = [file for file in glob.glob("Created\\Anonymisation\\text_rq2_names_matched_*.pkl")]

# Call in all files in the anonymisation folder
file_dict = {}
for file in filename_list:
    filename = open(file, "rb")
    f = pickle.load(filename)
    file_n = re.sub('.pkl', '', file)
    file_n = re.sub("Created\\\\Anonymisation\\\\text_rq2_names_matched_", '', file_n)
    print(file_n)
    file_dict[file_n] = f

# Checked for Review and Initial
text_cols = (['Child Social Work Assessment for Review Child Protection Conference_text',
       'Child Social Work Assessment to Initial Child Protection Conference_text',
       'Contact and Referral Form_text',
       'Child Social Work Assessment for Review Child Protection Conference_text_prev',
       'Child Social Work Assessment to Initial Child Protection Conference_text_prev',
        'Child Social Work Assessment_text', 'Child Social Work Assessment_text_prev',
       'Contact and Referral Form_text_prev']) 

col_wo_identifier_dict = {}
for text_col in text_cols:
    text_col_df = [df for df in file_dict.keys() if text_col in df]
    if 'prev' in text_col:
        text_col_df = [df for df in text_col_df if 'prev' in df]
    else:
        text_col_df = [df for df in text_col_df if 'prev' not in df]
    print(text_col_df)
    text_col_df_values = [v for k, v in file_dict.items() if k in text_col_df]
    print(len(text_col_df_values))
    col_wo_identifier_dict[text_col] = pd.concat(text_col_df_values, axis = 0)
    col_wo_identifier_dict[text_col].sort_index(inplace = True)

In [ ]:
from functools import reduce

# Deduplicate by index (in case any rows are repeated in different files)
for col, df in col_wo_identifier_dict.items():
    print(col)
    print(df.shape)
    col_wo_identifier_dict[col] = df.loc[~df.index.duplicated(keep = 'first')]
    print(col_wo_identifier_dict[col].shape)
    col_wo_identifier_dict[col].rename(col, inplace = True)    
    
# Merge together on index
df_wo_identifier_together = reduce(lambda left, right: pd.merge(left,right,left_index = True, right_index = True, how = 'outer'), col_wo_identifier_dict.values())
df_wo_identifier_together = df_wo_identifier_together.loc[~df_wo_identifier_together.index.duplicated(keep = 'first')]
print(sorted(df_wo_identifier_together.columns))

# Save knitted data all together 
with open("Created/text_rq2_names_matched.pkl", "wb") as handle:
    pickle.dump(df_wo_identifier_together, handle, protocol = pickle.HIGHEST_PROTOCOL)